In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/youtoxic_english_1000.csv")

df.head()


In [ ]:
# Dimensiones del dataset
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

df.info()

print("\nValores nulos por columna:")
print(df.isnull().sum())


In [ ]:
# Distribución del target IsToxic
sns.countplot(x="IsToxic", data=df)
plt.title("Distribución de comentarios tóxicos vs no tóxicos")
plt.xlabel("¿Es tóxico?")
plt.ylabel("Número de comentarios")
plt.show()

# Porcentaje de cada clase
toxicity_ratio = df["IsToxic"].value_counts(normalize=True) * 100
print("Distribución porcentual:\n", toxicity_ratio)


In [ ]:
# Crear columnas nuevas para análisis de texto
df["text_length_chars"] = df["Text"].apply(len)
df["text_length_words"] = df["Text"].apply(lambda x: len(x.split()))

# Mostrar estadísticos por tipo de comentario
df.groupby("IsToxic")[["text_length_chars", "text_length_words"]].describe()


In [ ]:
# Boxplots para comparar longitud de texto por clase
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(x="IsToxic", y="text_length_chars", data=df, ax=axes[0])
axes[0].set_title("Longitud en caracteres por tipo de comentario")

sns.boxplot(x="IsToxic", y="text_length_words", data=df, ax=axes[1])
axes[1].set_title("Longitud en palabras por tipo de comentario")

plt.tight_layout()
plt.show()


### Comparativa de longitud de los comentarios (Boxplot)

En esta visualización comparamos la **longitud de los comentarios tóxicos y no tóxicos**, tanto en número de caracteres como de palabras.

#### ¿Qué es un boxplot?
Un **boxplot** (o diagrama de caja) es una representación visual que nos permite ver:
- La **mediana** del conjunto de datos (línea central de la caja).
- El **rango intercuartílico** (donde se concentra el 50% de los valores).
- Los **valores extremos o outliers** (representados como puntos).

#### ¿Qué vemos en estos gráficos?
- La longitud de los comentarios **tóxicos y no tóxicos** es muy similar.
- Hay **muchos comentarios cortos** en ambos grupos.
- Aparecen **algunos comentarios muy largos** (outliers), especialmente en caracteres.
- No se observa una diferencia clara que nos permita decir que un tipo de comentario sea más largo que otro de forma sistemática.

#### ¿Qué concluimos?
Aunque puede haber pequeñas diferencias, **la longitud del comentario no parece ser un buen indicador por sí solo de si un comentario es tóxico o no**. Aun así, es útil conocer estas características para posibles decisiones de preprocesamiento, como filtrar comentarios excesivamente largos o cortos.


In [ ]:
# Filtrar comentarios tóxicos y no tóxicos
toxic_comments = df[df["IsToxic"] == True]["Text"]
nontoxic_comments = df[df["IsToxic"] == False]["Text"]

# Juntarlos en dos grandes textos para analizarlos
toxic_text = " ".join(toxic_comments).lower()
nontoxic_text = " ".join(nontoxic_comments).lower()


### Análisis del contenido textual

Para entender mejor las diferencias entre los comentarios tóxicos y no tóxicos, hemos separado los textos en dos grupos:

- Comentarios etiquetados como **tóxicos**.
- Comentarios etiquetados como **no tóxicos**.

Hemos unido los comentarios de cada grupo en un único texto para poder analizar qué palabras aparecen con mayor frecuencia en cada uno. Este enfoque nos permitirá visualizar patrones de lenguaje característicos, que luego pueden ser clave para entrenar un modelo predictivo.

En los próximos pasos generaremos:
- Listados de palabras más frecuentes.
- Nubes de palabras (*wordclouds*).
- N-gramas más comunes (combinaciones típicas de 2-3 palabras).


In [ ]:
from collections import Counter
import re

# Función para limpiar texto básico (sin lematizar aún)
def limpiar_texto(texto):
    texto = re.sub(r"[^\w\s]", "", texto)  # quitar signos de puntuación
    texto = texto.lower()  # pasar a minúsculas
    return texto

# Aplicar limpieza y dividir en palabras
palabras_toxicas = limpiar_texto(toxic_text).split()
palabras_no_toxicas = limpiar_texto(nontoxic_text).split()

# Contar palabras más comunes
frecuentes_toxicas = Counter(palabras_toxicas).most_common(10)
frecuentes_no_toxicas = Counter(palabras_no_toxicas).most_common(10)

# Mostrar resultados
print("🔴 Palabras más frecuentes en comentarios tóxicos:")
for palabra, freq in frecuentes_toxicas:
    print(f"{palabra}: {freq}")

print("\n🟢 Palabras más frecuentes en comentarios no tóxicos:")
for palabra, freq in frecuentes_no_toxicas:
    print(f"{palabra}: {freq}")


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

# Lista de stopwords en inglés
stop_words = set(stopwords.words("english"))

# Función mejorada que filtra stopwords
def limpiar_y_filtrar(texto):
    texto = re.sub(r"[^\w\s]", "", texto.lower())
    palabras = texto.split()
    return [p for p in palabras if p not in stop_words]

# Aplicar función mejorada
palabras_toxicas_filtradas = limpiar_y_filtrar(toxic_text)
palabras_no_toxicas_filtradas = limpiar_y_filtrar(nontoxic_text)

# Contar palabras más frecuentes (filtradas)
frecuentes_toxicas_filtradas = Counter(palabras_toxicas_filtradas).most_common(10)
frecuentes_no_toxicas_filtradas = Counter(palabras_no_toxicas_filtradas).most_common(10)

# Mostrar resultados
print("🔴 Palabras más frecuentes (tóxicos, sin stopwords):")
for palabra, freq in frecuentes_toxicas_filtradas:
    print(f"{palabra}: {freq}")

print("\n🟢 Palabras más frecuentes (no tóxicos, sin stopwords):")
for palabra, freq in frecuentes_no_toxicas_filtradas:
    print(f"{palabra}: {freq}")


### Análisis con eliminación de stopwords

Las palabras más frecuentes que observamos inicialmente eran muy comunes y poco informativas. Por ello, hemos repetido el análisis **eliminando las stopwords**, es decir, palabras muy frecuentes en inglés que no aportan significado real (como "the", "and", "is", etc.).

Esta limpieza **no forma aún parte del preprocesamiento oficial**, pero se introduce aquí como una forma de enriquecer el EDA y tomar decisiones más informadas.

Ahora los resultados empiezan a revelar **patrones de contenido más relevantes** para entender qué vocabulario podría distinguir los comentarios tóxicos de los no tóxicos.


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Unir las palabras ya filtradas
texto_toxico_filtrado = " ".join(palabras_toxicas_filtradas)
texto_nontoxico_filtrado = " ".join(palabras_no_toxicas_filtradas)

# Crear las nubes
wc_toxico = WordCloud(width=800, height=400, background_color="white").generate(texto_toxico_filtrado)
wc_nontoxico = WordCloud(width=800, height=400, background_color="white").generate(texto_nontoxico_filtrado)

# Mostrar
fig, axs = plt.subplots(1, 2, figsize=(18, 8))

axs[0].imshow(wc_toxico, interpolation="bilinear")
axs[0].axis("off")
axs[0].set_title("🔴 Comentarios tóxicos (sin stopwords)")

axs[1].imshow(wc_nontoxico, interpolation="bilinear")
axs[1].axis("off")
axs[1].set_title("🟢 Comentarios no tóxicos (sin stopwords)")

plt.tight_layout()
plt.show()


### WordClouds sin palabras vacías (stopwords)

Las siguientes nubes de palabras muestran los términos más repetidos en los comentarios **tóxicos** y **no tóxicos**, tras eliminar las palabras vacías típicas del inglés (como “the”, “and”, “is”…).

#### ¿Qué observamos?
- En los comentarios tóxicos aparecen con más frecuencia palabras como **“fuck”**, lo que indica un tono agresivo.
- También se observan términos raciales y relacionados con el orden público (**black, white, police**) en ambos grupos, lo que sugiere que el contexto es similar, pero el uso del lenguaje es lo que cambia.

Esta visualización permite a cualquier lector, incluso sin formación técnica, entender mejor el tipo de lenguaje que caracteriza cada grupo de comentarios.
